In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
#from torch.optim.lr_scheduler import StepLR
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import xarray as xr
#import joblib
#import pickle

In [10]:
import pandas as pd
import numpy as np
import datetime as dt
#import os
#import shutil
import seaborn as sns
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
def process_reppu_data():
    #path = r'/home/ryuho/Documents/reddy/research/SMRAI/Data/REPPU/200/pbig5min.dat' #Ubuntu
    path = r'/home/sachin/Documents/NIPR/Research/Data/REPPU/pbig5min.dat' #Server

    #read the REPPU data
    with open (path) as f:
        rectype = np.dtype(np.float32)
        reppu_data = np.fromfile(f, rectype) #size = 109,900,800

    reppu_data = reppu_data.reshape(-1,30,80)
    reppu_data = reppu_data.mean(axis=1).mean(axis=1) #integrate over the 30x80 grid

    #load MHD dates to match with REPPU data
    mhd_data = pd.read_csv('mhd_dates.csv')
    expanded_dt = pd.concat([pd.Series(pd.date_range(start, end)) 
        for start, end in zip(mhd_data['start'], mhd_data['end'])])
    
    time = np.arange(288)
    dt = []
    for day in expanded_dt:
        for t in time:
            dt.append(day + pd.Timedelta(minutes=t*5))
    dt = np.array(dt) #convert from list to numpy array
    df = pd.DataFrame(reppu_data, index=dt)
    df = df.rename(columns={0:'pot'})
    df.reset_index(inplace=True)
    df.rename(columns={'index':'dt'}, inplace=True)

    return df

reppu_df = process_reppu_data()
reppu_df   

,dt,pot
0,2021-05-10 00:00:00,5484.762695
1,2021-05-10 00:05:00,5247.855957
2,2021-05-10 00:10:00,5592.299805
3,2021-05-10 00:15:00,5834.336914
4,2021-05-10 00:20:00,5604.553711
...,...,...
45787,2022-08-19 23:35:00,-12768.093750
45788,2022-08-19 23:40:00,-13011.533203
45789,2022-08-19 23:45:00,-12235.481445
45790,2022-08-19 23:50:00,-9142.743164


In [12]:
def process_omni_and_merge(reppu_df):
    reppu_df = reppu_df.copy()
    
    omni_df = pd.read_csv('omni_mhd_5min.csv')
    omni_df = omni_df.interpolate()
    #omni_df = omni_df.ffill().bfill() #interpolate missing values
    #omni_df = omni_df.dropna() 
    omni_df['dt'] = pd.to_datetime(omni_df['dt'])

    #merge the REPPU and OMNI data
    df = pd.merge(reppu_df, omni_df, on='dt', how='outer')
    #df = df[(df['dt'] >= '2021-12-01') & (df['dt'] <= '2022-01-24')]
    #df = df[(df['dt'] >= '2021-12-01') & (df['dt'] <= '2021-12-02')]
    #df = df[(df['dt'] >= '2021-05-10') & (df['dt'] <= '2021-05-15')] #Kp 7 storm
    df['pot'] = df['pot'] * 1e-3
    #interpolate
    df = df.interpolate()

    return df.reset_index(drop=True)

df = process_omni_and_merge(reppu_df)
df

/tmp/ipykernel_5395/1113075550.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  omni_df = omni_df.interpolate()


,dt,pot,BY_GSE,BZ_GSE,flow_speed,proton_density,tilt_angle
0,2021-05-10 00:00:00,5.484763,4.720,6.240,344.0,12.1100,0.253032
1,2021-05-10 00:05:00,5.247856,4.350,6.550,344.0,11.4400,0.249749
2,2021-05-10 00:10:00,5.592300,4.610,6.560,345.0,12.6500,0.246494
3,2021-05-10 00:15:00,5.834337,4.650,6.890,344.0,12.6300,0.243269
4,2021-05-10 00:20:00,5.604554,4.880,7.340,345.0,12.4700,0.240075
...,...,...,...,...,...,...,...
45787,2022-08-19 23:35:00,-12.768094,-2.715,-4.450,653.5,6.3925,0.187198
45788,2022-08-19 23:40:00,-13.011534,-2.490,-4.470,655.0,6.2200,0.183724
45789,2022-08-19 23:45:00,-12.235482,-2.265,-4.555,654.0,5.9050,0.180298
45790,2022-08-19 23:50:00,-9.142744,-2.040,-4.640,653.0,5.5900,0.176873


In [13]:
def single_feat_df(df, lookback, var):
    df = df.copy()
    #df = df[['dt',var]]

    df.set_index('dt', inplace=True)
    
    for i in range(1, lookback+1):
        #df[f'{var}(t-{i})'] = df[var].shift(i)
        df[f'BY_GSE(t-{i})'] = df['BY_GSE'].shift(i)
        df[f'BZ_GSE(t-{i})'] = df['BZ_GSE'].shift(i)
        df[f'flow_speed(t-{i})'] = df['flow_speed'].shift(i)
        df[f'proton_density(t-{i})'] = df['proton_density'].shift(i)
        df[f'tilt_angle(t-{i})'] = df['tilt_angle'].shift(i)
        
    df.dropna(inplace=True)
    
    return df

feat = 'pot'
df = single_feat_df(df, 6, feat)
df

,pot,BY_GSE,BZ_GSE,flow_speed,proton_density,tilt_angle,BY_GSE(t-1),BZ_GSE(t-1),flow_speed(t-1),proton_density(t-1),...,BY_GSE(t-5),BZ_GSE(t-5),flow_speed(t-5),proton_density(t-5),tilt_angle(t-5),BY_GSE(t-6),BZ_GSE(t-6),flow_speed(t-6),proton_density(t-6),tilt_angle(t-6)
dt,,,,,,,,,,,,,,,,,,,,,
2021-05-10 00:30:00,5.467443,4.620,8.020,344.0,12.2600,0.233785,4.970,7.680,344.0,12.2100,...,4.350,6.550,344.0,11.4400,0.249749,4.720,6.240,344.0,12.1100,0.253032
2021-05-10 00:35:00,4.203517,4.260,8.380,345.0,12.6500,0.230692,4.620,8.020,344.0,12.2600,...,4.610,6.560,345.0,12.6500,0.246494,4.350,6.550,344.0,11.4400,0.249749
2021-05-10 00:40:00,3.692068,3.280,8.890,344.0,12.1200,0.227636,4.260,8.380,345.0,12.6500,...,4.650,6.890,344.0,12.6300,0.243269,4.610,6.560,345.0,12.6500,0.246494
2021-05-10 00:45:00,2.930368,6.480,5.390,345.0,13.9800,0.224618,3.280,8.890,344.0,12.1200,...,4.880,7.340,345.0,12.4700,0.240075,4.650,6.890,344.0,12.6300,0.243269
2021-05-10 00:50:00,2.470604,6.850,5.850,345.0,13.9900,0.221639,6.480,5.390,345.0,13.9800,...,4.970,7.680,344.0,12.2100,0.236913,4.880,7.340,345.0,12.4700,0.240075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-19 23:35:00,-12.768094,-2.715,-4.450,653.5,6.3925,0.187198,-2.940,-4.430,652.0,6.5650,...,-4.480,-2.840,654.0,6.2100,0.204643,-1.950,-1.660,642.0,6.5400,0.208167
2022-08-19 23:40:00,-13.011534,-2.490,-4.470,655.0,6.2200,0.183724,-2.715,-4.450,653.5,6.3925,...,-3.935,-3.615,651.5,6.5600,0.201133,-4.480,-2.840,654.0,6.2100,0.204643
2022-08-19 23:45:00,-12.235482,-2.265,-4.555,654.0,5.9050,0.180298,-2.490,-4.470,655.0,6.2200,...,-3.390,-4.390,649.0,6.9100,0.197622,-3.935,-3.615,651.5,6.5600,0.201133


In [14]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [15]:
def scale_split_tensor(df):

    X = df.iloc[:, 1:]
    y = df.iloc[:, 0]

    #scale the data
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    split_index = int(0.8*len(X))#80% train, 20% test

    #split the data
    X_train = X[:split_index]
    X_test = X[split_index:]
    y_train = y[:split_index]
    y_test = y[split_index:]

    #add exta dim for torch reqs
    X_train = X_train.reshape(-1, X.shape[1], 1)
    X_test = X_test.reshape(-1, X.shape[1], 1)
    y_train = y_train.values.reshape(-1, 1)
    y_test = y_test.values.reshape(-1, 1)

    #convert to tensor
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor

X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = scale_split_tensor(df)

In [16]:
X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape

(torch.Size([36628, 35, 1]),
 torch.Size([9158, 35, 1]),
 torch.Size([36628, 1]),
 torch.Size([9158, 1]))

In [17]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([16, 35, 1]) torch.Size([16, 1])


In [19]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, 
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

model = LSTM(1, 32, 2)
model.to(device)
model

LSTM(
  (lstm): LSTM(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)

In [20]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0
    
    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0

    print()

In [21]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0
    
    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)
    
    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [22]:
learning_rate = 0.001
num_epochs = 200
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

    #every 5 epochs plot the performance
    with torch.no_grad():
        predicted = model(X_train_tensor.to(device)).to('cpu')
        

Epoch: 1
Batch 100, Loss: 36.038
Batch 200, Loss: 37.592
Batch 300, Loss: 34.711
Batch 400, Loss: 38.109
Batch 500, Loss: 37.013
Batch 600, Loss: 37.324
Batch 700, Loss: 37.240
Batch 800, Loss: 34.243
Batch 900, Loss: 38.552
Batch 1000, Loss: 32.568
Batch 1100, Loss: 35.952
Batch 1200, Loss: 35.913
Batch 1300, Loss: 36.659
Batch 1400, Loss: 35.858
Batch 1500, Loss: 35.190
Batch 1600, Loss: 35.247
Batch 1700, Loss: 33.792
Batch 1800, Loss: 35.256
Batch 1900, Loss: 34.717
Batch 2000, Loss: 33.932
Batch 2100, Loss: 33.733
Batch 2200, Loss: 38.064

Val Loss: 42.234
***************************************************

Epoch: 2
Batch 100, Loss: 36.299
Batch 200, Loss: 33.953
Batch 300, Loss: 35.675
Batch 400, Loss: 33.741
Batch 500, Loss: 35.782
Batch 600, Loss: 35.420
Batch 700, Loss: 34.036
Batch 800, Loss: 32.206
Batch 900, Loss: 36.728
Batch 1000, Loss: 35.963
Batch 1100, Loss: 33.015
Batch 1200, Loss: 36.085
Batch 1300, Loss: 32.744
Batch 1400, Loss: 32.297
Batch 1500, Loss: 32.080
Batc

In [ ]:
X = df.iloc[:, 1:]
split_index = int(0.8*len(X))#80% train, 20% test
X_train = X[:split_index]

In [ ]:
with torch.no_grad():
    predicted = model(X_train_tensor.to(device)).to('cpu')

plt.figure(figsize=(10, 4))
y_true = y_train_tensor.to('cpu')
R = np.corrcoef(predicted.squeeze(), y_true.squeeze())[0,1]

plt.plot(X_train.index, y_true,  label=f'Actual {feat}')
plt.plot(X_train.index, predicted, label=f'Predicted {feat}')

plt.title(f'Prediction using LSTM from 21/05/10 -- 21/05/14, Epochs: {num_epochs}')
plt.annotate(f'R={R:.2f}', xy=(0.05, 0.1), xycoords='axes fraction', fontsize=12,fontweight='bold')
plt.xlabel(' ')
plt.ylabel(f'Polar Cap Potential [kV]')
plt.legend()
plt.grid()
plt.tight_layout()
#plt.savefig(f'{feat}_lstm_dt.png', dpi=300)
#plt.show()

In [ ]:
def train_and_validate(epochs, model, train_loader, test_loader, optimizer, loss_function, device):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train(True)  # Set the model to train mode
        print(f'Epoch: {epoch + 1}')
        running_loss = 0.0
        
        # Training loop
        for batch_index, batch in enumerate(train_loader):
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_index % 100 == 99:  # print every 100 batches
                avg_loss_across_batches = running_loss / 100
                print('Training Batch {0}, Loss: {1:.3f}'.format(batch_index+1, avg_loss_across_batches))
                running_loss = 0.0
        
        train_losses.append(running_loss / len(train_loader))

        # Validation loop
        model.train(False)  # Set the model to evaluation mode
        running_loss = 0.0
        
        for batch_index, batch in enumerate(test_loader):
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            
            with torch.no_grad():
                output = model(x_batch)
                loss = loss_function(output, y_batch)
                running_loss += loss.item()

        val_losses.append(running_loss / len(test_loader))
        
        print('Validation Loss: {0:.3f}'.format(val_losses[-1]))

    return train_losses, val_losses

#earning_rate = 0.001
#num_epochs = 10
#loss_function = nn.MSELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#train_losses, val_losses = train_and_validate(num_epochs, model, train_loader, test_loader, optimizer, loss_function, device)
